# Data Acquistion

The first phase of the project is to acquire all of the data that is needed for this project. The initial data required can be broken down into three separate data sets:

1. The FourSquare Top 30 Venues to Visit in Chicago
1. For each of the Top Site get a list of up restaurants in the surrounding area
1. The Chicago Police Department Crime Data for the last Year

---

# Import Libraries

In this section we import the libraries that will be required to process the data.

The first library is **Pandas**.  
Pandas is an open source, BSD-licensed library, providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language. Pandas will be used to house each of the data sets.

The second library is **Requests**.
Requests is a Python HTTP library, released under the Apache2 License. The goal of the project is to make HTTP requests simpler and more human-friendly.

The next library in **BeautifulSoup**
Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [1]:
# Import Pandas to provide DataFrame support
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import Requests
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

In [2]:
# Yaml is used to store some of the required configurations
import yaml

with open("./capstone_data/config.yaml", "r") as f:
    cfg = yaml.load(f)
 
search_params = {
    'client_id': cfg['client_id'],
    'client_secret': cfg['client_secret'],
    'intent': 'browse',
    'limit': 50,
    'v': cfg['version']
}

---

## FourSquare Top 30 Venues to Visit in Chicago

FourSquare does not actually provide an API that will return a list of the top venues to visit in a city. To get this list we can though use the FourSquare website directly to request the top sites in Chicago and then use BeautifulSoup to scrape the data we need. Once we have this starting data the other supplemental data we need to complete this dataset can be retrieved from using the FourSquare Venue API.

In [3]:
# Use the Requests get method to request the top sites in Chicago
page = requests.get(
    "https://foursquare.com/explore?mode=url&near=Chicago%2C%20IL%2C%20United%20States&nearGeoId=72057594042815334&q=Top%20Picks")

# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(page.content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')

A sample venue is is extracted from the `top_venues` list, formatted and shown below:

~~~~
<div class="venueDetails">
  <div class="venueName">
    <h2>
  <a href="/v/millennium-park/42b75880f964a52090251fe3" target="_blank">Millennium Park
  </a>
</h2>
  </div>
  <div class="venueMeta">
    <div class="venueScore positive" 
         style="background-color: #00B551;" 
         title="9.7/10 - People like this place">9.7</div>
    <div class="venueAddressData">
      <div class="venueAddress">201 E Randolph St (btwn Columbus Dr &amp; Michigan Ave), Chicago</div>
      <div class="venueData">
        <span class="venueDataItem">
          <span class="categoryName">Park</span>
          <span class="delim"> • </span>
        </span>
      </div>
    </div>
  </div>
</div>
~~~~

From this HTML the following data will be extracted:

* Venue Name
* Venue Score
* Venue Category 
* Venue HREF
* Venue ID (Extracted from the HREF)


### Create Top Venues Dataframe

The `top_venues` list, a sample of which is shown above, only contains some of the data required. In addition to the attributes extracted directly from the HTML code the following attributes are also required:
* Venue Address
* Venue Postalcode
* Venue City
* Venue Latitude
* Venue Longitude

These attributes will be obtained directly from FourSquare using the `venues` API. The process is as follows:
1. Create a new empty Pandas dataframe to hold the data for the Top Sites / Venues
1. Extract the available attributes from the HTML code
1. For each venue
    1. Contruct a URL to interagate the FourSquare Venue API for each top site
    1. Using the `venues` API and the URL request the data from FourSquare
    1. Get the properly formatted address and the latitude and longitude data from the returned JSON
    1. Write the data for each venue to the top venues dataframs

In [4]:
# The column names for the top venues dataframe
venue_columns = ['id', 
                 'score', 
                 'category', 
                 'name', 
                 'address',
                 'postalcode',
                 'city',
                 'href', 
                 'latitude', 
                 'longitude']

# Create the empty top venues dataframe
df_top_venues = pd.DataFrame(columns=venue_columns)

# For each venue in the BeautifulSoup HTML object
for venue in top_venues:
    
    # Extract the available attributes
    venue_name = venue.find(target="_blank").get_text()
    venue_score = venue.find(class_="venueScore positive").get_text()
    venue_cat = venue.find(class_="categoryName").get_text()
    venue_href = venue.find(class_="venueName").h2.a['href']
    venue_id = venue_href.split('/')[-1]

    if 'promotedTipId' in venue_id: 
        continue
        
    # Contruct the FourSquare venue API URL
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id, 
        cfg['client_id'],
        cfg['client_secret'],
        cfg['version'])
    
    # Request the venue data
    result = requests.get(url).json()
        
    # Get the properly formatted address and the latitude and longitude
    venue_address = result['response']['venue']['location']['address']
    venue_postalcode = result['response']['venue']['location']['postalCode']
    venue_city = result['response']['venue']['location']['city']
    venue_latitude = result['response']['venue']['location']['lat']
    venue_longitude = result['response']['venue']['location']['lng']
    
    # Add the venue to the top venues dataframe
    df_top_venues = df_top_venues.append({'id': venue_id,
                                          'score': venue_score,
                                          'category': venue_cat,
                                          'name': venue_name,
                                          'address': venue_address,
                                          'postalcode': venue_postalcode,
                                          'city': venue_city,
                                          'href': venue_href,
                                          'latitude': venue_latitude,
                                          'longitude': venue_longitude}, ignore_index=True)


In [5]:
# Verify the shape of the top venues dataframe
df_top_venues.shape

(30, 10)

In [6]:
# Verify the dtypes of the top venues dataframe
df_top_venues.dtypes

id             object
score          object
category       object
name           object
address        object
postalcode     object
city           object
href           object
latitude      float64
longitude     float64
dtype: object

In [7]:
# The score type needs to be converted to float
df_top_venues['score'] = pd.to_numeric(df_top_venues['score'], errors='coerce').fillna(0)

# Describe the score to see if there is nuch variance in the values
df_top_venues.score.describe()

count    30.000000
mean      9.526667
std       0.073968
min       9.400000
25%       9.500000
50%       9.500000
75%       9.600000
max       9.700000
Name: score, dtype: float64

In [8]:
# Review the head of the dataframe to make sure it looks as expected
df_top_venues.head()

,id,score,category,name,address,postalcode,city,href,latitude,longitude
0,42b75880f964a52090251fe3,9.7,Park,Millennium Park,201 E Randolph St,60601,Chicago,/v/millennium-park/42b75880f964a52090251fe3,41.882699,-87.623644
1,4b9511c7f964a520f38d34e3,9.6,Trail,Chicago Lakefront Trail,Lake Michigan Lakefront,60611,Chicago,/v/chicago-lakefront-trail/4b9511c7f964a520f38...,41.967053,-87.646909
2,49e9ef74f964a52011661fe3,9.6,Art Museum,The Art Institute of Chicago,111 S Michigan Ave,60603,Chicago,/v/the-art-institute-of-chicago/49e9ef74f964a5...,41.879665,-87.623630
3,4f2a0d0ae4b0837d0c4c2bc3,9.6,Deli / Bodega,Publican Quality Meats,825 W Fulton Market,60607,Chicago,/v/publican-quality-meats/4f2a0d0ae4b0837d0c4c...,41.886642,-87.648718
4,4aa05f40f964a520643f20e3,9.6,Theater,The Chicago Theatre,175 N State St,60601,Chicago,/v/the-chicago-theatre/4aa05f40f964a520643f20e3,41.885578,-87.627286


In [9]:
# Finally write the dataframe to a pickle file for restoring later
df_top_venues.to_pickle('./capstone_pickles/top_venues.pkl')

---

## FourSquare Restaurent Recommendations Data

Using the the list of all venue id values in the Top Sites DataFrame and the FourSquare categoryID that represents all food venues, we now search for restaurants within a 500 meter radius.



The requests returns a JSON object which can then be queried for the restaurant details required. A sample restaurnt from the results returned is shown below:

~~~~
{  
    "referralId":"v-1538424503",
    "hasPerk":"False",
    "venuePage":{  
        "id":"135548807"
    },
    "id":"55669b9b498ee34e5249ea61",
    "location":{  
        "labeledLatLngs":[  
            {  
                "label":"display",
                "lng":-87.62460021795313,
                "lat":41.88169538551873
            }
        ],
        "crossStreet":"btwn E Madison & E Monroe St",
        "postalCode":"60603",
        "formattedAddress":[  
            "12 S Michigan Ave (btwn E Madison & E Monroe St)",
            "Chicago, IL 60603",
            "United States"
        ],
        "distance":155,
        "city":"Chicago",
        "lng":-87.62460021795313,
        "neighborhood":"The Loop",
        "cc":"US",
        "state":"IL",
        "address":"12 S Michigan Ave",
        "lat":41.88169538551873,
        "country":"United States"
    },
    "name":"Cindy's",
    "categories":[  
        {  
            "pluralName":"Gastropubs",
            "id":"4bf58dd8d48988d155941735",
            "name":"Gastropub",
            "primary":"True",
            "icon":{  
                "prefix":"https://ss3.4sqi.net/img/categories_v2/food/gastropub_",
                "suffix":".png"
            },
            "shortName":"Gastropub"
        }
    ]
}
~~~~

From this JSON the following attributes are extraced and added to the Dataframe:

* Restaurant ID
* Restaurant Category Name
* Restaurant Category ID
* Restaurant Nest_name
* Restaurant Address
* Restaurant Postalcode
* Restaurant City
* Restaurant Latitude
* Restaurant Longitude
* Venue Name
* Venue Latitude
* Venue Longitude

The only piece of data that is missing is the Score or Rating of the Restaurant. To get this we need to make another FourSquare API query using the id of the Restaurant.

Using just the data in this DataFrame we will be able to generate maps displaying the chosen Top List Venue and the best scored surrounding restaurants.

In [10]:
# The column names for the restaurants dataframe
restaurants_columns = ['id',
                       'score', 
                       'category', 
                       'categoryID', 
                       'name', 
                       'address',
                       'postalcode',
                       'city',
                       'latitude',
                       'longitude', 
                       'venue_name', 
                       'venue_latitude',
                       'venue_longitude']

# Create the empty top venues dataframe
df_restaurant = pd.DataFrame(columns=restaurants_columns)

# Create a list of all the top venue latitude and longitude
top_venue_lats = df_top_venues['latitude'].values
top_venue_lngs = df_top_venues['longitude'].values

# Create a list of all the top venue names
top_venue_names = df_top_venues['name'].values

# Iterate over each of the top venues
# The venue name, latitude and longitude are passed to the loop
for ven_name, ven_lat, ven_long in zip(top_venue_names, top_venue_lats, top_venue_lngs):
    
    # Configure additional Search parameters
    # This is the FourSquare Category Id for all food venues
    categoryId = '4d4b7105d754a06374d81259'
    radius = 500
    limit = 50
    
    # Contruct the FourSquare search API URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
        cfg['client_id'],
        cfg['client_secret'],
        ven_lat,
        ven_long,
        cfg['version'],
        categoryId,
        radius,
        limit)
    
    # Make the search request
    results = requests.get(url).json()
    
    # Want a good selection of Restaurents
    # If less than 10 are returned ignore
    if len(results['response']['venues']) < 10:
        continue
        
    # Populate the new dataframe with the list of restaurants
    # Get the values for each Restaurant from the JSON
    for restaurant in results['response']['venues']:
 
        # Sometimes the Venue JSON is missing data. If so ignore and continue
        try:
            # Get location details
            rest_id = restaurant['id']
            rest_category = restaurant['categories'][0]['pluralName']
            rest_categoryID = restaurant['categories'][0]['id']
            rest_name = restaurant['name']
            rest_address = restaurant['location']['address']
            rest_postalcode = restaurant['location']['postalCode']
            rest_city = restaurant['location']['city']
            rest_latitude = restaurant['location']['lat']
            rest_longitude = restaurant['location']['lng']
            
            # Contruct the FourSquare venue API URL to get the venues rating / score
            rest_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
                rest_id, 
                cfg['client_id'],
                cfg['client_secret'],
                cfg['version'])

            # Get the restaurant score and href
            result = requests.get(rest_url).json()
            rest_score = result['response']['venue']['rating']
            
            # Add the restaurant details to the dataframe
            df_restaurant = df_restaurant.append({'id': rest_id,
                                                  'score': rest_score,
                                                  'category': rest_category,
                                                  'categoryID': rest_categoryID,
                                                  'name': rest_name,
                                                  'address': rest_address,
                                                  'postalcode': rest_postalcode,
                                                  'city': rest_city,
                                                  'latitude': rest_latitude,
                                                  'longitude': rest_longitude,
                                                  'venue_name': ven_name,
                                                  'venue_latitude': ven_lat,
                                                  'venue_longitude': ven_long}, ignore_index=True)
            
        # If there are any issue with a restaurant ignore and continue
        except:
            continue



In [11]:
# Verify the shape of the restaurants dataframe
df_restaurant.shape

(440, 13)

In [12]:
# Verify the dtypes of the restaurants dataframe
df_restaurant.dtypes

id                  object
score              float64
category            object
categoryID          object
name                object
address             object
postalcode          object
city                object
latitude           float64
longitude          float64
venue_name          object
venue_latitude     float64
venue_longitude    float64
dtype: object

In [13]:
# Review the head of the dataframe to make sure it looks as expected
df_restaurant.head()

,id,score,category,categoryID,name,address,postalcode,city,latitude,longitude,venue_name,venue_latitude,venue_longitude
0,4e73881c922e0a374024c7b8,7.7,Coffee Shops,4bf58dd8d48988d1e0931735,Starbucks,8 N. Michigan Avenue,60602,Chicago,41.882478,-87.624701,Millennium Park,41.882699,-87.623644
1,4ed3dd7e93ad987b5112d29f,8.2,Bakeries,4bf58dd8d48988d16a941735,Panera Bread,2 N Michigan Ave,60602,Chicago,41.882273,-87.624795,Millennium Park,41.882699,-87.623644
2,4e879cdc93adfd051d6d609e,9.2,Breakfast Spots,4bf58dd8d48988d143941735,Wildberry Pancakes & Cafe,130 E Randolph St,60601,Chicago,41.884599,-87.623203,Millennium Park,41.882699,-87.623644
3,4e56e50352b1d8d4e311da9e,9.0,Bakeries,4bf58dd8d48988d16a941735,Toni Patisserie & Café,65 E Washington St,60602,Chicago,41.883237,-87.625362,Millennium Park,41.882699,-87.623644
4,55669b9b498ee34e5249ea61,9.2,Gastropubs,4bf58dd8d48988d155941735,Cindy's,12 S Michigan Ave,60603,Chicago,41.881695,-87.624600,Millennium Park,41.882699,-87.623644


In [14]:
# Describe the score to see if there is nuch variance in the values
df_restaurant.score.describe()

count    440.000000
mean       8.120000
std        0.927711
min        5.300000
25%        7.600000
50%        8.300000
75%        8.800000
max        9.600000
Name: score, dtype: float64

In [15]:
# How many of the top 30 sites / venues had > 10 restaurants nearby
df_restaurant.venue_name.nunique()

11

In [16]:
# How many unique restaurant categories are there
df_restaurant.category.nunique()

71

In [17]:
# How many unique restaurants are there
df_restaurant.name.nunique()

222

In [18]:
# What arethe top 10 most frequently occuring restaurant types
df_restaurant.groupby('category')['name'].count().sort_values(ascending=False)[:10]

category
Coffee Shops                70
New American Restaurants    23
Pizza Places                22
American Restaurants        21
Italian Restaurants         19
Mexican Restaurants         17
Cafés                       16
Bakeries                    14
Sandwich Places             14
Donut Shops                 14
Name: name, dtype: int64

In [19]:
# Which restaurants have to highest average score
df_restaurant.groupby('category')['score'].mean().sort_values(ascending=False)[:10]

category
Seafood Restaurants                 9.480000
Chocolate Shops                     9.400000
Diners                              9.300000
Japanese Restaurants                9.300000
Gourmet Shops                       9.200000
Southern / Soul Food Restaurants    9.100000
Cuban Restaurants                   9.100000
Breakfast Spots                     9.071429
Mediterranean Restaurants           9.000000
Music Venues                        9.000000
Name: score, dtype: float64

In [20]:
# Finally write the dataframe to a pickle file for restoring later
df_restaurant.to_pickle('./capstone_pickles/restaurants.pkl')

---

# Import and process the Chicago Crime DataSet
This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago in the last year, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system. In order to protect the privacy of crime victims, addresses are shown at the block level only and specific locations are not identified.

| Column Name   | Type          | Description                                            | 
| :------------ | :------------ | :----------------------------------------------------- | 
| CASE#         | Plain Text    | The Chicago Police Department RD Number (Records Division Number), which is unique to the incident. | 
| DATE OF OCCURRENCE | Date & Time   | Date when the incident occurred. this is sometimes a best estimate. |
| BLOCK	        | Plain Text    | The partially redacted address where the incident occurred, placing it on the same block as the actual address. |
| IUCR	        | Plain Text    | The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e. |
| PRIMARY DESCRIPTION   | Plain Text    | The primary description of the IUCR code. |
| SECONDARY DESCRIPTION	| Plain Text    | The secondary description of the IUCR code, a subcategory of the primary description. |
| LOCATION DESCRIPTION | Plain Text | Description of the location where the incident occurred. |
| ARREST        | Plain Text    | Indicates whether an arrest was made. |
| DOMESTIC      | Plain Text    | Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act. |
| BEAT          | Plain Text    | Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74. |
| WARD	        | Number        | The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76. |
| FBI CD        | Plain Text    | Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html. |
| X COORDINATE	| Plain Text    | The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block. |
| Y COORDINATE	| Plain Text    | The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block. |
| LATITUDE	    | Number        | The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block. |
| LONGITUDE	    | Number        | The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block. |
| LOCATION	    | Location      | The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block. |
	




### Import the 2018 DataSet

The full dataset, 2001 to 2018 contains over 6.7M rows. This makes processing the dataset difficult and time consuming. Only the 2018 data, 187222 records, will be used. In addition, the following clean-up steps are required:
1. Not all of the columns are required. The following columns are removed:
    1. IUCR
    1. ARREST
    1. DOMESTIC
    1. BEAT
    1. WARD
    1. FBI CD
    1. X COORDINATE
    1. Y COORDINATE
    1. LOCATION

In [21]:
# These are the columns that we want to keep.
# Columns not listed here won't be imported, speeding things up.
crime_keep_columns = ['CASE#',
                      'DATE  OF OCCURRENCE',
                      'BLOCK', 
                      ' PRIMARY DESCRIPTION',
                      'WARD',
                      'LATITUDE',
                      'LONGITUDE']

In [22]:
# Download csv
# !wget -O './capstone_data/crimes.csv' https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv?accessType=DOWNLOAD

In [23]:
# Read cvs download into datafram
df = pd.read_csv('./capstone_data/crimes.csv',
                 usecols=crime_keep_columns)

Looking at the Shape of df we can see that only 7 columns have been imported instead of all 22.

In [24]:
df.shape

(264387, 7)

Let's have a quick look at the Head of the imported data

In [25]:
df.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,PRIMARY DESCRIPTION,WARD,LATITUDE,LONGITUDE
0,JB241987,04/28/2018 10:05:00 PM,009XX N LONG AVE,NARCOTICS,37.0,41.897895,-87.760744
1,JB241350,04/28/2018 08:00:00 AM,008XX E 53RD ST,CRIMINAL DAMAGE,5.0,41.798635,-87.604823
2,JB245397,04/28/2018 09:00:00 AM,062XX S MICHIGAN AVE,THEFT,20.0,41.780946,-87.621995
3,JB241444,04/28/2018 12:15:00 PM,046XX N ELSTON AVE,THEFT,39.0,41.965404,-87.736202
4,JB241667,04/28/2018 04:28:00 PM,022XX S KENNETH AVE,ARSON,22.0,41.850673,-87.735597


In [26]:
df.tail()

,CASE#,DATE OF OCCURRENCE,BLOCK,PRIMARY DESCRIPTION,WARD,LATITUDE,LONGITUDE
264382,JA525920,11/26/2017 02:02:00 PM,081XX S ASHLAND AVE,THEFT,21.0,41.746260,-87.663332
264383,JA475906,10/18/2017 01:31:00 PM,048XX N SHERIDAN RD,ASSAULT,46.0,41.969326,-87.654826
264384,JA534571,12/03/2017 03:18:00 AM,002XX W 38TH PL,BATTERY,3.0,41.824547,-87.633096
264385,JA492228,10/30/2017 04:00:00 PM,072XX S SOUTH SHORE DR,THEFT,7.0,41.764728,-87.561272
264386,JA536019,12/02/2017 08:00:00 PM,083XX S INDIANA AVE,CRIMINAL DAMAGE,6.0,41.742865,-87.619547


---
### Clean up the data and prepare

The sample data file was downloaded in the middle of September. So half of September of 2017 was missing and half of September 2018 was not yet populated. To simply resolve this the raw data was altered to move the September 2017 dates to September 2018. 

Now that the data has been imported it needs to be cleaned.
1. Move September 2017 dates to September 2018
1. Clean up the column names:
    1. Strip leading & trailing whitespace
    1. Replace multiple spaces with a single space
    1. Remove # characters
    1. Replace spaces with _
    2. Convert to lowercase
1. Change the date of occurance field to a date / time object
1. Add new columns for:
    1. Hour
    1. Day
    1. Month
    1. Year
    1. etc.
1. Split Block into zip_code and street
1. Verify that all rows have valid data

In [27]:
# Strip leading & trailing whitespace
df.columns = df.columns.str.strip()

# Replace multiple spaces with a single space
df.columns = df.columns.str.replace('\s{2,}', ' ')

# Replace # with blank
df.columns = df.columns.str.replace('#', '')

# Replace spaces with _
df.columns = df.columns.str.replace(' ', '_')

# Convert to lowercase
df.columns = df.columns.str.lower()

# Move September 2017 dates to September 2018
df.date_of_occurrence.replace(to_replace="(09/\\d+)/2017", value=r"\1/2018", regex=True, inplace=True)

In [28]:
# Verify that all datatype are as expected
df.dtypes

case                    object
date_of_occurrence      object
block                   object
primary_description     object
ward                   float64
latitude               float64
longitude              float64
dtype: object

Change the date of occurance field to a date / time object

In [29]:
df['date_of_occurrence'] =  pd.to_datetime(df['date_of_occurrence'], format='%m/%d/%Y %I:%M:%S %p')

Now that the date date of the crime is a real date time object let's add new columns for the hour, day, month and year of the crime etc:
1. Hour
1. Day Name
1. Day of week (Tuesday is the first day)
1. Month Name
1. Month Number
1. Year
1. Year and Month

In [30]:
# Add new columns to the dataframe to allow hourly, daily & monthly analysis
df['hour'] = df['date_of_occurrence'].dt.hour
df['day_name'] = df['date_of_occurrence'].dt.day_name()
df['day'] = df['date_of_occurrence'].dt.dayofweek + 1
df['month_name'] = df['date_of_occurrence'].dt.month_name()
df['month'] = df['date_of_occurrence'].dt.month
df['year'] = df['date_of_occurrence'].dt.year
df['year_month'] = df['date_of_occurrence'].dt.to_period('M')

The Block attribute can be split into zip_code and street

In [31]:
# Add the zip and street attributes
df['zip'] = df.block.str.split(' ').str[0]
df['street'] = df.block.str.split(' ').str[1:].apply(', '.join)

Verify that all rows have valid data

In [32]:
# Verify that all rows have valid data
df.isna().sum()

case                      0
date_of_occurrence        0
block                     0
primary_description       0
ward                      2
latitude               2378
longitude              2378
hour                      0
day_name                  0
day                       0
month_name                0
month                     0
year                      0
year_month                0
zip                       0
street                    0
dtype: int64

Some Latitude and Longitude values are missing, drop them.

In [33]:
# Drop rows with missing values 
df.dropna(inplace=True)

In [34]:
# Reindex
df.reset_index(inplace=True)

In [35]:
# Have a final look at the crime dataframe
df.head()

,index,case,date_of_occurrence,block,primary_description,ward,latitude,longitude,hour,day_name,day,month_name,month,year,year_month,zip,street
0,0,JB241987,2018-04-28 22:05:00,009XX N LONG AVE,NARCOTICS,37.0,41.897895,-87.760744,22,Saturday,6,April,4,2018,2018-04,009XX,"N, LONG, AVE"
1,1,JB241350,2018-04-28 08:00:00,008XX E 53RD ST,CRIMINAL DAMAGE,5.0,41.798635,-87.604823,8,Saturday,6,April,4,2018,2018-04,008XX,"E, 53RD, ST"
2,2,JB245397,2018-04-28 09:00:00,062XX S MICHIGAN AVE,THEFT,20.0,41.780946,-87.621995,9,Saturday,6,April,4,2018,2018-04,062XX,"S, MICHIGAN, AVE"
3,3,JB241444,2018-04-28 12:15:00,046XX N ELSTON AVE,THEFT,39.0,41.965404,-87.736202,12,Saturday,6,April,4,2018,2018-04,046XX,"N, ELSTON, AVE"
4,4,JB241667,2018-04-28 16:28:00,022XX S KENNETH AVE,ARSON,22.0,41.850673,-87.735597,16,Saturday,6,April,4,2018,2018-04,022XX,"S, KENNETH, AVE"


In [36]:
# Finally write the dataframe to a pickle file for restoring later
df.to_pickle('./capstone_pickles/crimes.pkl')

The data is now ready for visualisation.